In [1]:
import pandas as pd
import os
from transformers import AutoTokenizer
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

c:\Users\S00180\anaconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\S00180\anaconda3\envs\ml\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
# 載入資料
file_path = '../../proxmox_documents/pve_claude_translated.csv'
embedding_model = "BAAI/bge-base-zh-v1.5"

data = pd.read_csv(file_path)
# # 整理欄位，合併成完整描述
data['combined_text'] = data.apply(
    lambda row: f"路徑: {row['Path']}\n方法: {row['Method']}\n描述: {row['Description_zh']}\n參數: {row['Parameter_Details_zh']}",
    axis=1
)

# # 檢查整理後的數據格式
data[['Path', 'combined_text']].head()

,Path,combined_text
0,/api2/json/access,路徑: /api2/json/access\n方法: GET\n描述: 目錄索引\n參數: {}
1,/api2/json/access/domains,路徑: /api2/json/access/domains\n方法: GET\n描述: 驗證...
2,/api2/json/access/domains,路徑: /api2/json/access/domains\n方法: POST\n描述: 新...
3,/api2/json/access/domains/{realm},路徑: /api2/json/access/domains/{realm}\n方法: GET...
4,/api2/json/access/domains/{realm},路徑: /api2/json/access/domains/{realm}\n方法: PUT...


In [5]:
documents = [Document(doc,metadata={"page":index}) for index,doc in enumerate(data['combined_text'].to_list())]
documents[:5]

[Document(metadata={'page': 0}, page_content='路徑: /api2/json/access\n方法: GET\n描述: 目錄索引\n參數: {}'),
 Document(metadata={'page': 1}, page_content='路徑: /api2/json/access/domains\n方法: GET\n描述: 驗證網域索引\n參數: {}'),
 Document(metadata={'page': 2}, page_content='路徑: /api2/json/access/domains\n方法: POST\n描述: 新增驗證伺服器\n參數: {\'realm\': \'身分驗證網域 ID\', \'type\': \'網域類型\', \'acr-values\': \'指定授權伺服器被要求用於身分驗證請求的身分驗證上下文類別參考值\', \'autocreate\': \'如果用戶不存在則自動創建\', \'base_dn\': \'LDAP基礎域名\', \'bind_dn\': \'LDAP綁定域名\', \'capath\': \'CA證書存儲路徑\', \'case-sensitive\': \'用戶名區分大小寫\', \'cert\': \'客戶端證書路徑\', \'certkey\': \'客戶端證書密鑰路徑\', \'check-connection\': \'檢查與伺服器的綁定連接\', \'client-id\': \'OpenID客戶端ID\', \'client-key\': \'OpenID客戶端密鑰\', \'comment\': \'描述\', \'default\': \'使用此作為默認域\', \'domain\': \'AD域名\', \'filter\': \'用戶同步的LDAP過濾器\', \'group_classes\': \'群組的對象類別\', \'group_dn\': \'群組同步的LDAP基礎域名。如果未設置，將使用base_dn\', \'group_filter\': \'群組同步的LDAP過濾器\', \'group_name_attr\': \'表示群組名稱的LDAP屬性。如果未設置或未找到，將使用DN的第一個值作為名稱\', \'is

In [6]:
def count_tokens_per_page_transformers(documents, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    token_counts = {}
    for doc in documents:
        page_num = doc.metadata['page']
        token_count = len(tokenizer.encode(doc.page_content))
        token_counts[page_num] = token_count
    
    for page, count in sorted(token_counts.items()):
        print(f"Page {page}: {count} tokens")
        
    return token_counts

# 使用方法
token_counts = count_tokens_per_page_transformers(documents,embedding_model)

Token indices sequence length is longer than the specified maximum sequence length for this model (940 > 512). Running this sequence through the model will result in indexing errors


Page 0: 28 tokens
Page 1: 33 tokens
Page 2: 940 tokens
Page 3: 55 tokens
Page 4: 969 tokens
Page 5: 53 tokens
Page 6: 463 tokens
Page 7: 31 tokens
Page 8: 46 tokens
Page 9: 45 tokens
Page 10: 53 tokens
Page 11: 43 tokens
Page 12: 31 tokens
Page 13: 99 tokens
Page 14: 109 tokens
Page 15: 31 tokens
Page 16: 49 tokens
Page 17: 47 tokens
Page 18: 65 tokens
Page 19: 45 tokens
Page 20: 38 tokens
Page 21: 66 tokens
Page 22: 190 tokens
Page 23: 89 tokens
Page 24: 145 tokens
Page 25: 104 tokens
Page 26: 65 tokens
Page 27: 193 tokens
Page 28: 61 tokens
Page 29: 196 tokens
Page 30: 61 tokens
Page 31: 67 tokens
Page 32: 94 tokens
Page 33: 212 tokens
Page 34: 182 tokens
Page 35: 96 tokens
Page 36: 97 tokens
Page 37: 74 tokens
Page 38: 39 tokens
Page 39: 144 tokens
Page 40: 92 tokens
Page 41: 88 tokens
Page 42: 49 tokens
Page 43: 239 tokens
Page 44: 31 tokens
Page 45: 36 tokens
Page 46: 39 tokens
Page 47: 167 tokens
Page 48: 63 tokens
Page 49: 104 tokens
Page 50: 65 tokens
Page 51: 61 tokens
Page 52

In [7]:
index_path="../../faiss/api_index"
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs={'device': 'cpu'}, # 如果有 GPU 可以改為 'cuda'
    encode_kwargs={'normalize_embeddings': True}
)
if not os.path.exists(index_path):
    # 首次建立：使用 from_documents
    print("FAISS index 不存在快取，正在建立index")
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(index_path)
    print("FAISS index 建立完成")
else:
    # 已存在：直接載入
    vectorstore = FAISS.load_local(index_path, embeddings,allow_dangerous_deserialization=True)
    print("FAISS index 存在快取，載入成功")

C:\Users\S00180\AppData\Local\Temp\ipykernel_13088\2705819177.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(



FAISS index 存在快取，載入成功


In [18]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [52]:
query = "GET qemu all"
for i in retriever.invoke(query):
    print(i.page_content,end='\n\n')

路徑: /api2/json/nodes/{node}/capabilities/qemu
方法: GET
描述: QEMU 功能索引。
參數: {'node': '叢集節點名稱'}

路徑: /api2/json/nodes/{node}/qemu/{vmid}/status/start
方法: POST
描述: 啟動虛擬機
參數: {'node': '叢集節點名稱', 'vmid': '虛擬機的(唯一)ID', 'force-cpu': '使用給定的字串覆蓋QEMU的-cpu參數', 'machine': '指定QEMU機器', 'migratedfrom': '叢集節點名稱', 'migration_network': '用於遷移的(子)網路CIDR', 'migration_type': '預設情況下，遷移流量使用SSH隧道加密。在安全的完全私人網路上，可以禁用此功能以提高性能', 'skiplock': '忽略鎖定 - 只有root用戶可以使用此選項', 'stateuri': '某些命令從此位置保存/恢復狀態', 'targetstorage': '從源存儲到目標存儲的映射。僅提供單個存儲ID會將所有源存儲映射到該存儲。提供特殊值"1"將把每個源存儲映射到自身', 'timeout': '等待最大秒數'}

路徑: /api2/json/nodes/{node}/qemu
方法: GET
描述: 虛擬機器索引(依節點)。
參數: {'node': '集群節點名稱', 'full': '確定活動虛擬機的完整狀態'}

